# Relatório EP2 - Programação Concorrente e Paralela

**Alunos**: 
- Erick Rodrigues de Santana, NUSP: 11222008
- Francisco Eugênio Wernke, NUSP: 11221870
- Thiago Guerrero Balera, NUSP: 11275297
- Vinicius Pereira Ximenes Frota, NUSP: 11221967

**Professor**: Alfredo Goldman

**Monitores**: Elisa Silva e Luciana Marques

O relatório a seguir foi elaborado com base nas instruções do EP2 da Matéria MAC0219 - Programação Concorrente e Paralela.

## Executando o programa
Para executar o programa *mandelbrot* nas versões com MPI (Sequencial, OpenMP e Pthreads), adicionamos um argumento extra para definir o número de processos a serem criados (num_processes). Além disto, nas versões internamente paralelizadas, ainda existe o argumento de quantas threads serão criadas (num_threads). Antes de executar os comandos, é necessário gerar os binários desta forma:

```bash
cd src
make
```

Agora, pode-se executar as três formas do mandelbrot com MPI da seguinte forma:

```bash
mpirun -np <num_processes> mandelbrot_mpi
mpirun -np <num_processes> mandelbrot_mpi_omp <num_threads>
mpirun -np <num_processes> mandelbrot_mpi_pth <num_threads>
```

## Observações

Todas as execuções do programa foram feitas na Rede Linux usando 20 medições para cada tipo de execução.

In [5]:
# Bibliotecas importadas
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt

# Leitura de dados CSV
csv = pd.read_csv("../measurements/data.csv")
print(csv)

        tipo  processos  threads         tempo
0        mpi          1        1  24237.120152
1        mpi          1        1  24395.095348
2        mpi          1        1  24355.067730
3        mpi          1        1  24333.413601
4        mpi          1        1  24350.424290
..       ...        ...      ...           ...
970  mpi_omp         16       32   1168.375492
971  mpi_omp         16       32   1420.077801
972  mpi_omp         16       32   1414.572239
973  mpi_omp         16       32   1179.508924
974  mpi_omp         16       32   1176.087618

[975 rows x 4 columns]


## Organização dos dados

Para agrupar os dados coletados usamos a estrutura de dados de dicionário. A estrutura pode ser entendida assim:
```json
{
    seq: Pair<Média, Intervalo de confiança>,
    pth: [
        2^0 threads: Pair<Média, Intervalo de confiança>,
        2^1 threads: Pair<Média, Intervalo de confiança>,
        ... ,
        2^5 threads: Pair<Média, Intervalo de confiança>
    ],
    omp: ...Igual a pthread,
    mpi: [
        1 process: Pair<Média, Intervalo de confiança>,
        2 processes: Pair<Média, Intervalo de confiança>,
        ...
        16 processes: Pair<Média, Intervalo de confiança>
    ],
    mpi_pth: {
        1 process: [
            1 thread: Pair<Média, Intervalo de confiança>,
            2 threads: Pair<Média, Intervalo de confiança>,
            4 threads: Pair<Média, Intervalo de confiança>,
            ...
            32 threads: Pair<Média, Intervalo de confiança>
        ],
        2 processes: [
            ...Mesmo que para 1 processo
        ],
        ...
        16 processes: [
            ...Mesmo que para 1 processo
        ],
    },
    mpi_omp: { ...Mesmo que mpi_pth }
}
```
Onde *seqio* representa o tempo total e *seq* representa o tempo sem as operações de I/O e gerenciamento de memória da versão sequencial.

Como o intervalo de confiança é simétrico em torno da média, só armazenamos os limiares.
<!--  apagar depois-->

In [6]:
data = {"seq": (0,0), "pth" : [], "omp" : [], "mpi" : [], "mpi_pth" : dict(), "mpi_omp" : dict()}

processos = [2**i for i in range(5)]
tipos = ["mpi", "mpi_pth", "mpi_omp"]

for tipo in tipos[1:]:
    for processo in processos:
        data[tipo][processo] = []



## Intervalo de confiança

Para 95% de nível de confiança e n-1 (14) graus de liberdade, obtemos pela tabela da t-student o valor 2.1448 para z.


In [7]:
def confidence_interval(vector, mean):
    z = 2.1448
    n = len(vector)
    sigma = 0
    for i in vector:
        sigma += (i - mean)**2;

    sigma /= n-1;
    sigma = sqrt(sigma);
    return z * sigma/sqrt(n);

In [8]:
for i in range(5):
    vector = csv["tempo"][i*15:(i+1)*15]
    mean = sum(vector)/15
    pair = (mean, confidence_interval(vector, mean))
    data["mpi"].append(pair)

for i in range(5*15, len(csv), 15):
    tipo = csv["tipo"][i]
    processos = csv["processos"][i]
    
    vector = csv["tempo"][i:i+15]
    mean = sum(vector)/15
    pair = (mean, confidence_interval(vector, mean))
    data[tipo][processos].append(pair)


In [9]:
data

{'seq': (0, 0),
 'pth': [],
 'omp': [],
 'mpi': [(24340.326674733333, 74.62604191774881),
  (13094.284884066667, 70.34995500144774),
  (6841.701110133333, 82.62360100746305),
  (3642.6725706666666, 81.44316272438556),
  (1995.2112674666666, 75.7045419957342)],
 'mpi_pth': {1: [(24348.083607266668, 75.55687711742361),
   (16151.4556568, 83.0896209821448),
   (15695.39265626667, 105.42695854439043),
   (15605.376656666665, 96.15638124431102),
   (15590.817260800004, 78.71013986120725),
   (15583.383464800001, 73.27152111698403)],
  2: [(13026.887925466668, 82.91086932891363),
   (8527.199252466668, 81.76203936024427),
   (8428.322060800001, 86.65056504731804),
   (8367.128753666668, 85.89284605066572),
   (8429.708035866666, 85.69429238172097),
   (8384.241580933332, 86.70570115372348)],
  4: [(6876.0444322, 77.66861966519785),
   (3702.7171612, 69.06500455247945),
   (2036.5857918, 81.91292538976612),
   (1369.5397853333336, 57.522759797702),
   (1342.9332732666664, 62.924147627474404),

## Visualização dos resultados

Para visualizar os resultados, plotamos um gráfico para cada tipo de programa em cada tipo de imagem e, nas versões paralelizáveis, para cada número de *threads* também.

Por motivos de melhor visualização dos resultados, optamos por exibir os eixos em escala logarítimica com base 2. O eixo x representa os expoentes do tamanho de entrada e o eixo y representa o log2 da média do tempo de execução em milissegundos.

In [ ]:
# Definição de fontes para o plot
axis_font = Plots.font("Helvetica", 7);
text_font = Plots.font("Helvetica", 10);

In [ ]:
# Definição do eixo x
x = [];
for i in 4:13
    append!(x, i);
end

In [ ]:
# Coleta dos dados necessários para cada plot
function fetch_data(figura, id)
    plot_seq = [];
    ci_seq = [];
    plot_seqio = [];
    ci_seqio = [];
    plot_pth = [];
    ci_pth = [];
    plot_omp = [];
    ci_omp = [];
    for i in 4:13
        append!(plot_seq, data["seq"][figura][2^i].first);
        append!(plot_seqio, data["seqio"][figura][2^i].first);
        append!(plot_pth, data["pth"][figura][2^i][id].first);
        append!(plot_omp, data["omp"][figura][2^i][id].first);

        append!(ci_seq, data["seq"][figura][2^i].second);
        append!(ci_seqio, data["seqio"][figura][2^i].second);
        append!(ci_pth, data["pth"][figura][2^i][id].second);
        append!(ci_omp, data["omp"][figura][2^i][id].second);
    end
    
    for i in 1:length(plot_seq)
        ci_seq[i] = log2(plot_seq[i] + ci_seq[i]) - log2(plot_seq[i]);
        ci_seqio[i] = log2(plot_seqio[i] + ci_seqio[i]) - log2(plot_seqio[i]);
        ci_pth[i] = log2(plot_pth[i] + ci_pth[i]) - log2(plot_pth[i]);
        ci_omp[i] = log2(plot_omp[i] + ci_omp[i]) - log2(plot_omp[i]);
    end
    
    plot_seq = broadcast(log2, plot_seq);
    plot_seqio = broadcast(log2, plot_seqio);
    plot_pth = broadcast(log2, plot_pth);
    plot_omp = broadcast(log2, plot_omp);
    
    return plot_seq, ci_seq, plot_seqio, ci_seqio, plot_pth, ci_pth, plot_omp, ci_omp;
end;

In [ ]:
# Plota os 4 gráficos de cada tipo de programa para cada imagem usando o número de threads fornecido
function plot_by_number_of_threads(threads)
    id = convert(Int64, log2(threads)) + 1;
    keys = ["full", "seahorseValley", "elephantValley", "tripleSpiralValley"];
    for figura in keys
        plot_seq, ci_seq, plot_seqio, ci_seqio, plot_pth, ci_pth, plot_omp, ci_omp = fetch_data(figura, id);
        
        title = "\n" * string(threads) * " Thread(s) - " * figura;
        
        seq = plot(x, plot_seq, label= "Seq sem I/O", lw = 1, legend=:bottomright,
            color="#7a5306", fillcolor="#f4b22e",
            seriestype = :scatter,
            ribbon = ci_seq)
        
        seqio = plot(x, plot_seqio, label= "Seq com I/O", lw = 1, legend=:bottomright,
            color="#810ee0", fillcolor="#d382ab",
            seriestype = :scatter,
            ribbon = ci_seqio)
        
         pth = plot(x,  plot_pth, label= "Pth", lw = 1, legend=:bottomright,
            color="#000396", fillcolor="#07ffc0",
            seriestype = :scatter,
            ribbon = ci_pth)
        
         omp = plot(x, plot_omp, label= "OMP", lw = 1, legend=:bottomright,
            color="#f60000", fillcolor="#29a7e1",
            seriestype = :scatter,
            ribbon = ci_omp)
        
        final_plot = plot(seq, seqio, pth, omp, size=(750, 410), layout=4,
            guidefont=axis_font, xtickfont=axis_font, ytickfont=axis_font,
            titlefont=text_font, legendfont = axis_font,
            bottom_margin = 7mm,
            xticks = [minimum(x):maximum(x)+1;],
            title=title, xlabel="Tamanho da entrada (log2)", ylabel="Tempo (log2(ms))")
        
        display(final_plot)
    end
end;


In [ ]:
plot_by_number_of_threads(2^0)

In [ ]:
plot_by_number_of_threads(2^1)

In [ ]:
plot_by_number_of_threads(2^2)

In [ ]:
plot_by_number_of_threads(2^3)

In [ ]:
plot_by_number_of_threads(2^4)

In [ ]:
plot_by_number_of_threads(2^5)

## Resultados

### Como e por que as três versões do programa se comportam com a variação:


* **Do tamanho da entrada?**
    
    O tempo gasto é proporcional ao tamanho da entrada independentemente da versão. Além disto, podemos ver que todos os gráficos possuem o mesmo formato mas com valores diferentes. Isto ocorre pois, em qualquer método escolhido para fazer a computação, um aumento no tamanho da entrada implica em mais trabalho a ser realizado e todos os métodos reagem de forma similar à este aumento. 
        
        
* **Das regiões do Conjunto de Mandelbrot?**
    
     O tempo gasto na figura *full* é relativamente menor do que o tempo gasto nas outras figuras na versão sequencial. Isto ocorre pois, em relação ao limitante *escape_radius_squared* definido, os pixels da figura *full* divergem em menos iterações do que os pixels das outras figuras, fazendo com que o programa execute mais rapidamente. Nas versões paralelizadas, a diferença de tempo gasto entre as figuras é relativa à quantidade de threads.
        
        
* **Do número de threads?**
    
    Conforme o número de threads aumenta, conseguimos observar, para tamanhos cada vez maiores de entrada, uma redução do tempo cada vez mais proporcional ao número de threads. Isto ocorre pois, quando começamos a ter tamanhos de entrada cada vez maiores, o benefício do uso das threads e da divisão do trabalho acabam superando o seu custo de criação.
    
    Além disso, observamos que, nas versões paralelizadas, quanto maior o número de threads, menor é a diferença de tempo gasto entre as imagens. Isto ocorre pois quanto mais dividimos a imagem em diferentes seções, mais homogênea fica a distribuição dos pixels que demoram muitas iterações para divergir e, portanto, as seções demoram relativamente o mesmo tempo. Assim o programa executa em menos tempo, pois mais threads executarão e terminarão juntas.
 
### Qual o impacto das operações de I/O e alocação de memória no tempo de execução?

Analisando o código, podemos perceber que as operações de I/O e alocação de memória gastam tempo proporcional a t^2 (O(t^2)), onde t é o tamanho da entrada. Podemos notar, então, que a influência dessas operações no tempo é relativa entre os pontos e, para valores de entrada pequenos, o tempo de processamento é ínfimo enquanto a maior parte do tempo gasto pela execução do programa é realizando essas operações.